In [7]:
import pandas as pd

df = pd.read_csv('data/life/internet.csv')
df = df.drop(columns=['합계출산율'])
df = df.dropna()
df.head(), df.shape, df['지역별'].unique()


(    지역별    시점  인터넷이용률
 204  강원  2012    72.1
 205  경기  2012    83.1
 206  경남  2012    73.2
 207  경북  2012    69.0
 208  광주  2012    81.4,
 (202, 3),
 array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '울산', '인천',
        '전남', '전북', '제주', '충남', '충북', '충청'], dtype=object))

In [8]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# '인터넷이용률'을 숫자형으로 변환 (문자열에서 숫자로)
yearly_data['인터넷이용률'] = pd.to_numeric(yearly_data['인터넷이용률'], errors='coerce')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'인터넷이용률': 'mean'}).round(3)
    
    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime인지 확인 후, 형식 변경
    if pd.api.types.is_datetime64_any_dtype(monthly_df['시점']):
        monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')
    
    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)
final_monthly_df['인터넷이용률'] = final_monthly_df['인터넷이용률'].map(lambda x: f"{x:.2f}")

# 결과 확인
print(final_monthly_df.head(12), final_monthly_df.shape, final_monthly_df['지역별'].unique())

# 데이터 저장
final_monthly_df.to_csv('data/life/인터넷이용률(월별).csv', index=False)

         시점 인터넷이용률 지역별
0   2012.01  72.10  강원
1   2012.02  72.41  강원
2   2012.03  72.72  강원
3   2012.04  73.02  강원
4   2012.05  73.33  강원
5   2012.06  73.64  강원
6   2012.07  73.95  강원
7   2012.08  74.26  강원
8   2012.09  74.57  강원
9   2012.10  74.88  강원
10  2012.11  75.18  강원
11  2012.12  75.49  강원 (2237, 3) ['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '울산' '인천' '전남' '전북' '제주'
 '충남' '충북' '충청']
